# Pseudo-Labeling Flow

This notebook implements an automated pseudo-labeling pipeline designed to streamline the annotation process for object detection and instance segmentation tasks. The tool iteratively improves model performance by using an initial model trained on a small set of manually annotated data to generate labels on new images, which can then be refined and used to retrain progressively better models.

**A "flow" represents a complete pseudo labeling run with specific configuration settings (model type, initial dataset size, correction strategy), while "iterations" are the individual training cycles within each flow where new data is added and the model is retrained.**

The features within this notebook include:
- **Automated Pipeline**: Complete workflow from data preparation to model training
- **Database Logging**: Database tracking for all iterations
- **CVAT Integration**: For viewing and adjusting annotations
- **Flexible Configuration**: Supports different model architectures and training settings
- **Status Monitoring**: Real-time pipeline status and progress tracking



Automatic Export to CVAT only tested and functional for Instance Segmentation + Object Detection

## Imports

In [4]:
import onedl.client

from pseudo_labeling import PseudoLabelingPipeline

## Global Initalizers
Configure the pipeline with your project-specific settings:


In [7]:
pipeline = PseudoLabelingPipeline(
    project_name="daniel-osman---streamlining-annotation-bootstrapping/testing",
    main_dataset_name="full-data:0", #input only
    initial_annotated_dataset_name="initial-annotations:0",
    validation_dataset="val:0",
    sample_size_per_iter=150,
    current_flow = 0,
    min_confidence=0.1,
    local_path='/Users/daniel/Documents/2025 BEP - VBTI Data/testing',
    cvat_project_id=88,
    db_path="pseudo_labeling_metadata.db"
)

print("Pipeline initialized")


2025-07-10 08:55:55.287 | INFO     | onedl._local_store.datasets:pull:838 - Pulling dataset full-data:0 from remote='daniel-osman---streamlining-annotation-bootstrapping/testing' with pull_policy=missing.
2025-07-10 08:55:55.293 | INFO     | onedl._local_store.datasets:pull:858 - Dataset full-data:0 already exists in local store. Skipping
2025-07-10 08:55:55.326 | INFO     | onedl._local_store.datasets:pull:838 - Pulling dataset initial-annotations:0 from remote='daniel-osman---streamlining-annotation-bootstrapping/testing' with pull_policy=missing.
2025-07-10 08:55:55.329 | INFO     | onedl._local_store.datasets:pull:858 - Dataset initial-annotations:0 already exists in local store. Skipping


Resuming incomplete iteration 1 (status: MERGE_COMPLETE)
GLOBAL INITIALIZATIONS INITIALIZED
Project: daniel-osman---streamlining-annotation-bootstrapping/testing
Main dataset: full-data:0
Initial annotated dataset: initial-annotations:0
Sample size per iteration: 150
Selected flow: f0
Initial annotated dataset contains: 50 samples
Flow f0 already exists in database - ready to resume
Last completed iteration: 0


2025-07-10 08:55:55.843 | INFO     | onedl._local_store.datasets:resolve_latest_version:582 - Resolved latest version of dataset train-f0 to 5 with remote='daniel-osman---streamlining-annotation-bootstrapping/testing'.
2025-07-10 08:55:55.844 | INFO     | onedl._local_store.datasets:load:375 - Resolved latest version of dataset train-f0 to 5.
2025-07-10 08:55:55.845 | INFO     | onedl._local_store.datasets:pull:838 - Pulling dataset train-f0:5 from remote='daniel-osman---streamlining-annotation-bootstrapping/testing' with pull_policy=missing.
2025-07-10 08:55:55.849 | INFO     | onedl._local_store.datasets:pull:858 - Dataset train-f0:5 already exists in local store. Skipping


Training dataset train-f0 exists with 50 samples
Ready for iteration 1

🔄 Attempting auto-recovery...
🔄 Recovering state for f0 iteration 1...
✓ Recovered predicted_dataset_name: pseudo-f0-0--cpu--266a4:0
✓ Recovered manual_corrections_mode: False
✓ Recovery complete - ready to resume
Pipeline initialized


## Training Config
Set up your model training parameters interactively:


In [12]:
# Option 1: Interactive widget setup (uncomment to use)
# train_cfg = pipeline.setup_training_config()


# Option 2: Direct dictionary configuration (recommended for specific config)
pipeline.train_cfg = {
    'model_type': 'FasterRCNNConfig',
    'task_type': 'object_detection',
    'backbone': 'RESNET_50',
    'epochs': 1,
    'batch_size': 3,
}

print("Training configuration set:")
print(pipeline.train_cfg)


Training configuration set:
{'model_type': 'FasterRCNNConfig', 'task_type': 'object_detection', 'backbone': 'RESNET_50', 'epochs': 1, 'batch_size': 3}


# (1) Initial Flow, Training, and Evaluation Setup

This step initiates the current flow and establishes the baseline model using your initial annotated dataset.
1. Loads the initial annotated dataset.
2. Created training set for the current flow.
3. Trains the first baseline model (iteration 0) for the specified flow.
4. Evaluates Model
5. Logs Metadata (Only variables generated throughout the process of the pipeline are 'predicted_dataset_name', 'model_uid', 'evaluation_uid', 'evaluation_info')

⚠️ ATTENTION: Skip this section if your current flow already exists and if you already have a baseline model

In [ ]:
pipeline.get_pipeline_status()

### 1.1 Train Initial Model and Evaluate on Validation Set
Train the baseline model on your initial annotated dataset.
Evaluate the initial model performance on the validation dataset.




If you want to use an existing model, then run: <span style="color:#d73a49; font-family:monospace;">pipeline.log_iteration_0_external_model("smoky-shepherd-0")</span>, then skip to section (2):


In [ ]:
pipeline.train_model()
pipeline.evaluate_model()

### 1.2 Log
Save all metadata for the initial training iteration to the database.


In [ ]:
pipeline.log_iteration_0()
print("Initial model training and evaluation complete, Current status:")

# (2) Pseudo-Labeling Iteration Workflow

This step executes a complete pseudo-labeling iteration cycle using the model from the previous iteration to generate labels on new data.
1. Sets up the next iteration with correction strategy (manual or automated).
2. Samples new unlabeled data from the full dataset.
3. Runs inference using the previous iteration's model to generate pseudo-labels.
4. Handles corrections based on strategy: exports to CVAT for manual corrections OR merges pseudo-labels directly.
5. Trains updated model on expanded dataset (original + new data).
6. Evaluates the updated model performance.
7. Logs iteration metadata to track progress and results.

**⚠️ ATTENTION: Set 'manual_corrections=True' for CVAT workflow with human review, or 'manual_corrections=False' for fully automated pseudo-labeling**


In [8]:
pipeline.get_pipeline_status()


PIPELINE STATUS REPORT
Flow ID: f0
Current Iteration: 1
Current Status: MERGE_COMPLETE
Training Dataset: train-f0
Current Model UID: None
Training Configuration: None
Database Path: pseudo_labeling_metadata.db
Sample Size Per Iteration: 150
Minimum Confidence Threshold: 0.1

RECENT ITERATIONS:
  Iteration 1: MERGE_COMPLETE
  Iteration 0: COMPLETED (completed: 2025-07-09 12:08:24)


### Local Initializer
Configure the next iteration parameters

In [9]:
# Set manual_corrections=True for CVAT human review
# Set manual_corrections=False for fully automated pseudo-labeling
manual_corrections = False
pipeline.setup_next_iteration(manual_corrections)


Current iteration 1 status: MERGE_COMPLETE
Continuing with current iteration...
----------------------------------------
ITERATION INITIALIZED - PERSISTENT ARCHITECTURE
Flow ID: f0
Current iteration: 1
Manual corrections: False
Sample size this iteration: 150
GT added this iteration: 0
Pseudo added this iteration: 150
Total GT images after this step: 50
Total pseudo-labeled images after this step: 150
Total expected training set size: 200
Train dataset name: train-f0
Persistent pseudo dataset: pseudo-f0
Manual corrections dataset: manual-corrections-f0
Pseudo input dataset: pseudo-f0
Initial annotations: initial-annotations:0
Inference model UID: teal-ellipsis-0
----------------------------------------


### 2.1 Sample New Data

In [ ]:
pipeline.sample_unseen_inputs()


### 2.2 Generate Predictions/Pseudo-Labels


In [ ]:
# pipeline.set_inference_model_uid('')
# pipeline.set_predicted_dataset('broccoli-semantic-segmentation-part4-may23-train-0--cpu--75080')

pipeline.run_inference()

### 2.3 CVAT Export
Run even if manual correction is false.

In [ ]:
if pipeline.manual_corrections_global:
    print("Manual corrections enabled - proceeding to CVAT export")
    pipeline.manually_correct_cvat()
    print("After completing corrections in CVAT, manually update the predicted dataset and run the merge cell below")
else:
    print("No Manual Correction, Proceed to merging the datasets")


### 2.4. Merge Data
This cell merges the dataset with current training set. If **manual_correction = True**, then corrected annotations will be exported and merged.


In [10]:
pipeline.merge_pseudo_labels()

2025-07-10 08:56:55.188 | INFO     | onedl._local_store.datasets:pull:838 - Pulling dataset initial-annotations:0 from remote='daniel-osman---streamlining-annotation-bootstrapping/testing' with pull_policy=missing.
2025-07-10 08:56:55.191 | INFO     | onedl._local_store.datasets:pull:858 - Dataset initial-annotations:0 already exists in local store. Skipping


Starting simplified merge process...

=== AUTO PSEUDO-LABELING MODE ===
✓ Pseudo dataset already updated after inference

=== REBUILDING TRAINING DATASET (SIMPLIFIED) ===
✓ Started with initial dataset: 50 images


2025-07-10 08:56:55.647 | INFO     | onedl._local_store.datasets:resolve_latest_version:576 - There is no remote version. Resolved latest version of dataset manual-corrections-f0 to 0 local='daniel-osman---streamlining-annotation-bootstrapping/testing'
2025-07-10 08:56:55.648 | INFO     | onedl._local_store.datasets:load:375 - Resolved latest version of dataset manual-corrections-f0 to 0.
2025-07-10 08:56:55.648 | INFO     | onedl._local_store.datasets:pull:838 - Pulling dataset manual-corrections-f0:0 from remote='daniel-osman---streamlining-annotation-bootstrapping/testing' with pull_policy=missing.
2025-07-10 08:56:55.649 | INFO     | onedl._local_store.datasets:pull:849 - Pulling dataset manual-corrections-f0:0 from remote. pull_policy=<PullPolicy.missing: 'missing'> and dataset not found in local store.


✓ No manual corrections found for this flow: This resource cannot be found. GET https://api.onedl.ai/v2/storage/contexts/daniel-osman---streamlining-annotation-bootstrapping/testing/-/datasets/manual-corrections-f0:0/info
404 Not Found - {'detail': 'Dataset manual-corrections-f0:0 was not found in project daniel-osman---streamlining-annotation-bootstrapping/testing.'}
Received Body b'{"detail":"Dataset manual-corrections-f0:0 was not found in project daniel-osman---streamlining-annotation-bootstrapping/testing."}'


2025-07-10 08:56:56.200 | INFO     | onedl._local_store.datasets:resolve_latest_version:582 - Resolved latest version of dataset pseudo-f0 to 0 with remote='daniel-osman---streamlining-annotation-bootstrapping/testing'.
2025-07-10 08:56:56.200 | INFO     | onedl._local_store.datasets:load:375 - Resolved latest version of dataset pseudo-f0 to 0.
2025-07-10 08:56:56.201 | INFO     | onedl._local_store.datasets:pull:838 - Pulling dataset pseudo-f0:0 from remote='daniel-osman---streamlining-annotation-bootstrapping/testing' with pull_policy=missing.
2025-07-10 08:56:56.203 | INFO     | onedl._local_store.datasets:pull:858 - Dataset pseudo-f0:0 already exists in local store. Skipping
2025-07-10 08:56:56.210 | INFO     | onedl._local_store.datasets:pull:838 - Pulling dataset initial-annotations:0 from remote='daniel-osman---streamlining-annotation-bootstrapping/testing' with pull_policy=missing.
2025-07-10 08:56:56.211 | INFO     | onedl._local_store.datasets:pull:858 - Dataset initial-annot

✓ Added pseudo dataset: 150 images, total: 200


Generating label map from unique labels: 100%|██████████| 200/200 [00:00<00:00, 15570.21it/s]
2025-07-10 08:56:56.255 | INFO     | onedl.datasets.columns.base_column:_generate_label_map_from_unique_labels:457 - Generated label map: {0: 'AboveGround', 1: 'Defect', 2: 'Overgrown', 3: 'Stone', 4: 'Tip'}
2025-07-10 08:56:56.266 | INFO     | onedl._local_store.datasets:save:179 - Saved dataset train-f0:6 to local store.
2025-07-10 08:56:56.267 | INFO     | onedl._local_store.datasets:resolve_latest_version:567 - Resolved latest version of dataset train-f0 to 6 local='daniel-osman---streamlining-annotation-bootstrapping/testing'
2025-07-10 08:56:56.316 | INFO     | onedl._local_store.datasets:push:419 - Pushing 250 blobs to remote.
Files Confirmed:   0%|          | 0/251 [00:00<?, ?file/s]

Getting upload links:   0%|          | 0/251 [00:00<?, ?file/s]

Getting upload links: 100%|██████████| 251/251 [00:00<00:00, 891.54file/s]

Files Confirmed: 100%|█████████▉| 250/251 [00:00<00:00, 875.38f


✓ TRAINING DATASET REBUILT:
  - Initial GT: 50 images
  - Pseudo labels: 150 images
  - Total: 200 images
  - Saved as: train-f0
  - Label map: {0: 'AboveGround', 1: 'Defect', 2: 'Overgrown', 3: 'Stone', 4: 'Tip'}


### 2.5 Train Updated Model
Train a new model on the expanded training set

In [19]:
pipeline.train_model()

✓ Training already completed. Recovered model UID: insulated-aggregate-0


### 2.6 Evaluate Performance

In [ ]:
pipeline.evaluate_model()

### 2.7 Status

In [ ]:
pipeline.get_pipeline_status()

# Additional Runs
To run additional iterations, repeat Section 2 after logging. For creating a new flow, go back to Section 1, update the current_flow and go again.
